# INSTALACION DE LIBRERIAS

In [ ]:
import pandas as pd
from rapidfuzz import process,fuzz

# DEFINICIÓN DE RUTAS



In [ ]:
path_web_orders = 'data/Web_orders.txt'
path_catalog_orders = 'data/Catalog_Orders.txt'
path_products = 'data/products.txt'

# FUNCIONES

In [ ]:
def resumen_estadistico(df, nombre_dataset):
    print(f"--- RESUMEN ESTADÍSTICO: {nombre_dataset} ---")

    print("\n1. INFORMACIÓN GENERAL Y TIPOS:")
    print(df.info())

    print("\n2. ESTADÍSTICAS NUMÉRICAS:")
    print(df.describe())

    print("\n3. VARIANZA (Numeric only):")
    print(df.select_dtypes(include=['number']).var())

    print("\n4. ESTADÍSTICAS CATEGÓRICAS (Unicidad y Frecuencia):")
    print(df.describe(include=['object']))

    print("\n5. VALORES ÚNICOS (Cardinalidad):")
    print(df.nunique())

    print("\n6. VALORES NULOS:")
    print(df.isnull().sum())
    print("-" * 50 + "\n")

## Correción de delimitador
DATASET: web_orders

Hay 2 opciones:


*   Cambiar el delimitador del encabezado de "," a ";"
*   Cambiar todos los delimitadores de ";" a "," y asegurarse que la columna que contiene "," dentro de sus registros esten entre comillas doble

Se opta por la opción 1, ya que involucra menos pasos y menos iteración de las filas.

In [ ]:
with open(path_web_orders, 'r', encoding='utf-8') as f:
    lineas = f.readlines()

In [ ]:
lineas[0] = lineas[0].replace(',', ';')

In [ ]:
path_new_web_orders = 'Web_orders_csv.txt'
with open(path_new_web_orders, 'w', encoding='utf-8') as f:
    f.writelines(lineas)

# VALIDACIÓN DE FORMATO

## Verificación de formato
Se verifica archivo por archivo, si la cantidad de comillas dobles son par, sino, se debe corregir.

In [ ]:
paths = [path_new_web_orders,path_catalog_orders,path_products]

In [ ]:
for path in paths:
    print(f"\nVerificando archivo: {path}")
    try:
        with open(path, 'r', encoding='utf-8') as f:
            for i, linea in enumerate(f, start=1):
                num_comillas = linea.count('"')
                if num_comillas % 2 != 0:
                    print(f"Línea {i} con número impar de comillas ({num_comillas}):")
                    print(f"    {linea.strip()}")
    except Exception as e:
        print(f"Error leyendo {path}: {e}")

## Correción de formato (comilla faltante)
DATASET: catalog_orders

ID: 124

In [ ]:
path_new_catalog_orders = 'catalog_orders_csv.txt'
with open(path_catalog_orders, 'r', encoding='utf-8') as f:
    lineas = f.readlines()
lineas[124] = '124,139856.00,8/97/29 00:00:00,"","TY4400",1.00,"1353"\n'
with open(path_new_catalog_orders, 'w', encoding='utf-8') as f:
    f.writelines(lineas)

## Importación de dataset y definición de delimitadores
Como el dataset web_orders le cambiamos el delimitador, lo definimos en esta línea de código.



In [ ]:
df_web_orders = pd.read_csv(path_new_web_orders, sep=';')
df_catalog_orders = pd.read_csv(path_new_catalog_orders, sep=',')
df_products = pd.read_csv(path_products, sep=',')

# RESUMEN ESTADÏSTICO

In [ ]:
resumen_estadistico(df_products, "Products")
resumen_estadistico(df_catalog_orders, "Catalog Orders")
resumen_estadistico(df_web_orders, "Web Orders")

# ESTANDARIZACIÓN - web_orders

## Estandarización de columnas

*   Las columnas DATE, PCODE, CATALOG no estan en su columna correspondiente.
*   La columna custnum esta en minusculas y el contenido de la fila no esta acorde con el contenido de sus registros.

In [ ]:
df_web_orders.rename(columns = {'DATE':'PCODE','CATALOG':'DATE','PCODE':'CATALOG','custnum':'CUSTNAME'},inplace=True)

In [ ]:
df_web_orders.head()

In [ ]:
df_web_orders.shape

## Conversión de tipo de dato

### Analisis

In [ ]:
df_web_orders.dtypes

In [ ]:
df_web_orders.info()

### Conversión

In [ ]:
df_web_orders['INV']= df_web_orders['INV'].astype('Int64')
df_web_orders['QTY']= df_web_orders['QTY'].astype('Int64')
df_web_orders['PCODE']= df_web_orders['PCODE'].astype('string')
df_web_orders['CATALOG']= df_web_orders['CATALOG'].astype('string')
df_web_orders['CUSTNAME']= df_web_orders['CUSTNAME'].astype('string')
df_web_orders['DATE'] = pd.to_datetime(df_web_orders['DATE'], errors='coerce')

In [ ]:
df_web_orders.head()

### Verificación

In [ ]:
df_web_orders.dtypes

# SANITIZACIÓN - web_orders

## Verificación de nulos

In [ ]:
df_web_orders.info()

In [ ]:
df_web_orders.isna().sum()

Se encuentran valores nulos en los ID: 628, 778, 785

In [ ]:
df_web_orders[df_web_orders['QTY'].isna()]

Al ser solo 3 valores, elegí eliminar los nulos.

In [ ]:
df_web_orders = df_web_orders.dropna()

## Normalización de datos

### Normalización de PCODE
Hay 119 valores distintos de PCODE, pero no todos son correctos porque hay faltas ortograficas y errores de tipeo.

In [ ]:
df_web_orders['PCODE'].value_counts()

Se usará el dataset de products para poder estandarizar los valores de PCODE

In [ ]:
df_products.head()

Verificación de valores faltantes

In [ ]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

Se pondrá todo en mayuscula y se reemplaza la O por 0

In [ ]:
df_web_orders['PCODE'] = df_web_orders['PCODE'].str.upper()
df_web_orders['PCODE'] = df_web_orders['PCODE'].str.replace('O', '0', regex=False)

In [ ]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

In [ ]:
df_products['PCODE'].unique()

Se utiliza la libreria rapidfuzz para hacer una estimación de similitud de los datos faltantes

In [ ]:
referencia = df_products['PCODE'].dropna().astype(str).unique()
for valor in valores_faltantes:
    resultado = process.extractOne(valor, referencia, score_cutoff=60)
    print(f"{valor} → {resultado}")


Reemplazo en función de los resultados anteriores

In [ ]:
reemplazos = {
    'TY1900': 'TY1100',
    'GD10)!': 'GD1001',
    'GD12))': 'GD1200'
}

In [ ]:
df_web_orders['PCODE'] = df_web_orders['PCODE'].replace(reemplazos)

Verificación final de la normalización de la columna PCODE

In [ ]:
valores_faltantes = df_web_orders.loc[~df_web_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

### Normalización de CATALOG

In [ ]:
df_web_orders['CATALOG'].value_counts()

Se identifican las categorias más frecuentes para reasignar los errores en esas categorias

In [ ]:
conteo = df_web_orders['CATALOG'].value_counts()
catalogos_frecuentes = conteo[conteo > 50].index.tolist()
print(catalogos_frecuentes)

Se utliza la similitud con la libreria rapidfuzz para estimar la similitud de datos, tomo una similitud superior al 60%

In [ ]:
reemplazos = {}
for valor in df_web_orders['CATALOG'].unique():
    if valor in catalogos_frecuentes:
        continue
    mejor_match = process.extractOne(valor, catalogos_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 60:
        reemplazos[valor] = mejor_match[0]
print(reemplazos)

In [ ]:
df_web_orders['CATALOG'] = df_web_orders['CATALOG'].replace(reemplazos)
df_web_orders['CATALOG'].value_counts()


### Normalización de CUSTNAME

In [ ]:
df_web_orders['CUSTNAME'].value_counts()

In [ ]:
conteo = df_web_orders['CUSTNAME'].value_counts()
clientes_frecuentes = conteo[conteo > 10].index.tolist()
print(clientes_frecuentes)

No se ve ningún patron o posibles nombres mal escritos

In [ ]:
reemplazos = {}
for valor in df_web_orders['CUSTNAME'].unique():
    if valor in clientes_frecuentes:
        continue  # Si el valor lo encuentra, no se hace nada
    mejor_match = process.extractOne(valor, clientes_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 80:  # Umbral que hace que los que tengan 90% de similitud o más
        reemplazos[valor] = mejor_match[0]
print(reemplazos)

# ESTANDARIZACIÓN - catalog_orders

### Descripción de dataset

In [ ]:
df_catalog_orders.head()

In [ ]:
df_catalog_orders.info()

### Renombre de columna



In [ ]:
df_catalog_orders.rename(columns = {'custnum':'CUSTNUM'},inplace=True)

### Estandarizacón de QTY

In [ ]:
df_catalog_orders['QTY'].value_counts()

Busco los valores vacios y como aún esta considerado como String, cualquier cadena vacia.

In [ ]:
df_catalog_orders[df_catalog_orders['QTY'].isna() | (df_catalog_orders['QTY'].astype(str).str.strip() == '')]

Ya que la columna tiene valores en string como 1.0, 2.0, entre otros, primero debo convertirlo a decimal y luego recien a entero.

In [ ]:
df_catalog_orders['QTY'] = pd.to_numeric(df_catalog_orders['QTY'], errors='coerce')
df_catalog_orders['QTY']= df_catalog_orders['QTY'].astype('Int64')

Uso la misma validación anterior, pero ahora la parte que deberia ser la que devuelva es la funcion de "isna"

In [ ]:
df_catalog_orders[df_catalog_orders['QTY'].isna() | (df_catalog_orders['QTY'].astype(str).str.strip() == '')]

In [ ]:
df_catalog_orders.info()

### Conversión de tipo de dato

In [ ]:
df_catalog_orders['INV']= df_catalog_orders['INV'].astype('Int64')
df_catalog_orders['PCODE']= df_catalog_orders['PCODE'].astype('string')
df_catalog_orders['CATALOG']= df_catalog_orders['CATALOG'].astype('string')
df_catalog_orders['CUSTNUM']= df_catalog_orders['CUSTNUM'].astype('Int64')
df_catalog_orders['DATE'] = pd.to_datetime(df_catalog_orders['DATE'],format='%m/%y/%d %H:%M:%S', errors='coerce')

### Estandarización CATALOG

#### Descripción de valores

In [ ]:
df_catalog_orders['CATALOG'].value_counts()

Obtengo los catalogos más frecuentes, observando el conteo anterior, puedo poner un limite de mayor a 100

In [ ]:
conteo = df_catalog_orders['CATALOG'].value_counts()
catalogos_frecuentes = conteo[conteo > 100].index.tolist()
print(catalogos_frecuentes)

#### Estimación por similitud

In [ ]:
reemplazos = {}
for valor in df_catalog_orders['CATALOG'].dropna().unique():
    val_str = str(valor).strip()

    if val_str in catalogos_frecuentes:
        continue
    if val_str.isupper():
        matches = [c for c in catalogos_frecuentes if c.lower() == val_str.lower()]
        if matches:
            reemplazos[val_str] = matches[0]
            continue
    mejor_match = process.extractOne(val_str, catalogos_frecuentes, scorer=fuzz.token_sort_ratio)
    if mejor_match and mejor_match[1] >= 60:
        reemplazos[val_str] = mejor_match[0]
print(reemplazos)

#### Verificación de catalog

In [ ]:
df_catalog_orders['CATALOG'] = df_catalog_orders['CATALOG'].replace(reemplazos)
df_catalog_orders['CATALOG'].value_counts()

#### Valores nulos de catalog

In [ ]:
df_catalog_orders[df_catalog_orders['CATALOG'].isna() | (df_catalog_orders['CATALOG'].astype(str).str.strip() == '')]

In [ ]:
df_catalog_orders.info()

### Estandarización de PCODE

#### Descripción de PCODE

In [ ]:
df_catalog_orders[df_catalog_orders.isna().any(axis=1)]

In [ ]:
df_catalog_orders['PCODE'].value_counts()

#### Valores faltantes

In [ ]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

#### Correción de valores

In [ ]:
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].str.upper()
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].str.replace('O', '0', regex=False)

#### Verificación posterior

In [ ]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

#### Estimación por similitud de valores faltantes y reemplazo

In [ ]:
for valor in valores_faltantes:
    resultado = process.extractOne(valor, referencia, score_cutoff=60)
    print(f"{valor} : {resultado}")

In [ ]:
reemplazos = {
    'TY1900': 'TY1100',
    'CC220': 'CC2200',
    'GD10!)': 'GD1010',
    'GD10)!': 'GD1001',
    'GD12))':'GD1200',
    'CC43))':'CC4300',
    'C3200':'CC3200'
}

In [ ]:
df_catalog_orders['PCODE'] = df_catalog_orders['PCODE'].replace(reemplazos)

In [ ]:
valores_faltantes = df_catalog_orders.loc[~df_catalog_orders['PCODE'].isin(df_products['PCODE']), 'PCODE'].unique()
print(valores_faltantes)

### Imputación de datos de columna CATALOG mapeando la columna PCODE

In [ ]:
df_catalog_orders[df_catalog_orders['CATALOG'].isna() | (df_catalog_orders['CATALOG'].astype(str).str.strip() == '')]

#### Mapeo de valores de PCODE con su respectivo CATALOG

In [ ]:
pcode_catalog_map = df_catalog_orders[df_catalog_orders['CATALOG'].notna()].drop_duplicates(subset=['PCODE']) \
                      .set_index('PCODE')['CATALOG']

In [ ]:
print(pcode_catalog_map)

In [ ]:
catalog_faltantes = df_catalog_orders['CATALOG'].isna()
df_catalog_orders.loc[catalog_faltantes, 'CATALOG'] = df_catalog_orders.loc[catalog_faltantes, 'PCODE'].map(pcode_catalog_map)

#### Verificación de imputación

In [ ]:
df_catalog_orders[df_catalog_orders['ID'].isin([124, 2681, 5685])]

### Estandarización de CUSTNUM

#### Descripción de CUSTNUM

Se encontro valores dentro de CUSTNUM que no son correctos ya que es en 3004 registros solo son una secuencia de numeros que no representan al código del cliente. Ya que un código de INV debe tener solo un CUSTNUM porque es quien ha comprado el producto

In [ ]:
df_catalog_orders['CUSTNUM'].value_counts()

In [ ]:
custnum_counts = df_catalog_orders['CUSTNUM'].value_counts()
custnum_unicos = custnum_counts[custnum_counts == 1].index

In [ ]:
map_data = df_catalog_orders[~df_catalog_orders['CUSTNUM'].isin(custnum_unicos)]

#### Mapeo de INV y su respectivo CUSTNUM

In [ ]:
inv_to_cust_map = map_data.groupby('INV')['CUSTNUM'].agg(lambda x: x.value_counts().idxmax())
print(inv_to_cust_map)

In [ ]:
df_catalog_orders['CUSTNUM'].value_counts()

Para cuestiones de seguridad, creo una columna adicional de CUSTNUM ORIGINAL y comprar los valores con el valore reemplazado, tambien esto me ayuda a saber cuantos valores han sido reemplazados

In [ ]:
df_catalog_orders['CUSTNUM_ORIGINAL'] = df_catalog_orders['CUSTNUM']

In [ ]:
custnum_a_reemplazar = df_catalog_orders['CUSTNUM'].isin(custnum_unicos)

In [ ]:
df_catalog_orders.loc[custnum_a_reemplazar, 'CUSTNUM'] = df_catalog_orders.loc[custnum_a_reemplazar, 'INV'].map(inv_to_cust_map)

Se cambiaron 3004 registros, por el mapeo de INV con su CUSTNUM

In [ ]:
cambios = df_catalog_orders[df_catalog_orders['CUSTNUM'] != df_catalog_orders['CUSTNUM_ORIGINAL']][['ID', 'INV', 'CUSTNUM_ORIGINAL', 'CUSTNUM']]
print(cambios)

In [ ]:
df_catalog_orders.shape

df_catalog_orders.info()

#### Eliminación de valores nulos

Aún tenemos nulos en la columna de QTY, es posible realizar la imputación de datos en función a la Media, o moda en función a su PCODE, pero no sería tan preciso, por lo que al ser solo 6 valores de un total de 6767, se eliminarán.
Tambien elimino la columna CUSTNUM_ORIGINAL que creé para verificación

In [ ]:
df_catalog_orders = df_catalog_orders.dropna()
df_catalog_orders.drop(columns=['CUSTNUM_ORIGINAL'],inplace=True)

### Exportación de Dataset limpios

In [ ]:
df_catalog_orders.to_csv('catalog_orders_clean.csv', index=False, quoting=1)

In [ ]:
df_web_orders.to_csv('web_orders_clean.csv', index=False, quoting=1)

## ESTRATEGIA DE INTEGRACIÓN

### Eliminación los ID de los dataset ya limpios

In [ ]:
df_web_orders = df_web_orders.drop(columns=['ID'], errors='ignore')
df_catalog_orders = df_catalog_orders.drop(columns=['ID'], errors='ignore')

### Agregado de columna ORIGEN

Se agrega una columna "ORIGEN" para poder rastrear y contabilizar el origen de los datos.

In [ ]:
df_web_orders['ORIGEN'] = 'WEB'
df_catalog_orders['ORIGEN'] = 'CATALOG'

### Integrado de los dataset

In [ ]:
df_orders = pd.concat([df_web_orders, df_catalog_orders], ignore_index=True)

### Creación de un ID generalizado para el dataset integrado

In [ ]:
df_orders.insert(0, 'ID', range(1, len(df_orders) + 1))

### Verificación de nuevo dataset

In [ ]:
df_orders['ORIGEN'].value_counts()

In [ ]:
df_orders.head()

### Exportación de datos

Se exportan los dataset para luego importarlos al WEKA

In [ ]:
df_orders.to_csv('all_orders.csv', index=False, quoting=1)

In [ ]:
df_products.to_csv('products.csv', index=False, quoting=1)